In [1]:
import sys, os; sys.path.append('..')
import pyzx as zx
import random
import math
from fractions import Fraction
%config InlineBackend.figure_format = 'svg'

In [2]:
c = zx.qasm("""
qreg q[3];
cx q[0], q[1];
""")
zx.d3.draw(c)

In [3]:
c = zx.qasm("""
qreg q[2];
rx(0.5*pi) q[1];
t q[0];
cx q[0], q[1];
cx q[1], q[0];
cx q[0], q[1];
tdg q[1];
rx(-0.5*pi) q[0];
""")
zx.d3.draw(c)

In [4]:
c.gates

[XPhase(1,phase=1/2),
 T(0),
 CNOT(0,1),
 CNOT(1,0),
 CNOT(0,1),
 T*(1),
 XPhase(0,phase=-1/2)]

In [5]:
g = c.to_graph()
g

Graph(14 vertices, 15 edges)

In [6]:
zx.simplify.spider_simp(g)
zx.d3.draw(g)

spider_simp: 1. 1.  2 iterations


In [7]:
zx.full_reduce(g)
zx.d3.draw(g)

In [48]:
def t_optimiser(c):
    g = c.to_graph()
    g = zx.simplify.teleport_reduce(g)
    c_opt = zx.Circuit.from_graph(g).split_phase_gates().to_basic_gates()
    return zx.optimize.full_optimize(c_opt).to_basic_gates()

In [49]:
c = zx.Circuit.load('../circuits/Fast/mod5_4_before')
zx.d3.draw(c.to_graph())
print(c.to_basic_gates().stats())

Circuit mod5_4_before on 5 qubits with 63 gates.
        28 is the T-count
        35 Cliffords among which 
        28 2-qubit gates and 6 Hadamard gates.


In [50]:
c1 = t_optimiser(c)
zx.d3.draw(c1.to_graph())
print(c1.to_basic_gates().stats())

Circuit  on 5 qubits with 27 gates.
        8 is the T-count
        19 Cliffords among which 
        14 2-qubit gates and 2 Hadamard gates.


In [51]:
c.verify_equality(c1)

True

In [52]:
c2 = c1.copy()
c2.add_circuit(c.adjoint())
g = c2.to_graph()
zx.simplify.full_reduce(g)
zx.d3.draw(g)

In [53]:
c1.gates[10] = zx.gates.T(4, adjoint=True)

In [54]:
c.verify_equality(c1)

False

In [55]:
c2 = c1.copy()
c2.add_circuit(c.adjoint())
g = c2.to_graph()
zx.simplify.full_reduce(g)
zx.d3.draw(g)

In [39]:
g = zx.Circuit.load('../circuits/Fast/hwb6.qc').to_graph()
zx.d3.draw(g)
print(zx.tcount(g))

105


In [40]:
zx.simplify.full_reduce(g)
zx.d3.draw(g)
print(zx.tcount(g))

75


In [41]:
g.apply_state("++---+-")
g.apply_effect("+011-1-")
zx.simplify.full_reduce(g)
print(zx.tcount(g))
zx.drawing.arrange_scalar_diagram(g)
zx.d3.draw(g)

33


In [42]:
def compute_decomp(g):
    if zx.tcount(g) >= 6:
        gsum = zx.simulate.replace_magic_states(g)
        gsum.reduce_scalar()
        terms = 0
        vals = 0
        for g1 in gsum.graphs:
            t,v = compute_decomp(g1)
            terms += t
            vals += v
        return (terms, vals)
    else:
        return (2 ** math.ceil(zx.tcount(g)/2), g.to_matrix())

In [44]:
zx.simulate.max_terms_needed(g)
#math.ceil(2**(0.468 * zx.tcount(g)))

67228

In [45]:
compute_decomp(g)

(8865, array([[-0.0625+0.j]]))

In [ ]:
(2**12, zx.simulate.calculate_path_sum(g))